In [1]:
%cd ../src

/Users/mikhail_zybin/food-order-chat-bot/src


In [2]:
import chatbot
import openai

In [3]:
def read_scenario(scenario_id: int) -> list[dict[str, str]]:
    with open(f"../evaluator_scenarios/scenario{scenario_id}.txt", "r") as fin:
        scenario = fin.readlines()

    # This is to remove the commentaries
    while not scenario[0].startswith("Chatbot"):
        scenario = scenario[1:]
    
    messages = []
    current_replica = "Please help me to order food"
    chatbot_speaks = True
    for line in scenario:
        line = line.lstrip()
        if line.startswith("User"):
            messages.append({"role": "assistant", "content": current_replica.strip()})
            chatbot_speaks = False
            current_replica = line[5:]
        elif line.startswith("Chatbot"):
            messages.append({"role": "user", "content": current_replica.strip()})
            chatbot_speaks = True
            current_replica = line[9:]
        else:
            current_replica += line
    
    return messages

In [4]:
system_prompt = chatbot.initialize_system_prompt()

In [5]:
scenario_id = 1
messages = read_scenario(1)

In [6]:
api_base = "https://llama3-1-8b-api.llm.lab.epam.com/v1"
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
client = openai.OpenAI(api_key="***REMOVED***", base_url=api_base)


In [7]:
with open("prompts/evaluator_prompt.txt") as fin:
    evaluator_prompt = fin.read()

In [9]:
messages

[{'role': 'user', 'content': 'Please help me to order food'},
 {'role': 'assistant',
  'content': 'Hello, Dear Customer!\nI am food order chat bot! My function is to help you choose and order food from various cafes and restaurants.\nWhat would you like to eat today?'},
 {'role': 'user', 'content': 'pasta'},
 {'role': 'assistant',
  'content': 'Pasta is a great choice. Based on your preference, I would recommend the following restaurants that serve pasta:\n1. Roman Holiday (Italian restaurant)\n2. Sukhothai (Thai restaurant, they have a dish called Pad See Ew which is a stir-fried wide rice noodles with broccoli, egg, and your choice of protein in a sweet soy sauce)\nWould you like to choose one of these restaurants or would you like to explore other options?\nAlso, please let me know if you have any specific type of pasta in mind (e.g. spaghetti, fettuccine, etc.) or any specific toppings you would like to have.'},
 {'role': 'user',
  'content': 'What kinds of pasta do they have in Ro

In [8]:
def evaluate_ai_reply(
    template: str,
    messages: list[dict[str, str]],
    predicted_message, 
    ground_truth,
    model: str,
    client: openai.OpenAI,    
    temperature: float = 0.2,
) -> str:
    evaluator = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": template.format(str(messages), predicted_message, ground_truth)}
        ],
        temperature=temperature,
    )
    return evaluator.choices[0].message.content

In [ ]:
from loguru import logger

logger.disable("chatbot")
for i in range(4, len(messages), 2):
    predicted_message = chatbot.get_next_ai_message(messages[:i], model, client)
    ground_truth = messages[i]["content"]
    print("-"*20 + "predicted_message" + "-"*20)
    print(predicted_message)
    print("-"*20 + "ground_truth" + "-"*20)
    print(ground_truth)
    print()
    print(evaluate_ai_reply(evaluator_prompt, messages[:i], predicted_message, ground_truth, model, client))